Import All The Necessary Dependencies.

In [5]:
import numpy as numpy
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saqla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# loading the file into pandas 
twitter = pd.read_csv(r'D:\Brainwave Matrix/task-2/tweets.csv', encoding='ISO-8859-1')

In [9]:
type(twitter)

pandas.core.frame.DataFrame

In [10]:
# Rows and Columns
twitter.shape

(1599999, 6)

In [11]:
twitter.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
twitter.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [13]:
# name the columns 
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter = pd.read_csv(r'D:\Brainwave Matrix/task-2/tweets.csv', names = column_names, encoding='ISO-8859-1')

In [14]:
twitter.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
twitter.shape

(1600000, 6)

In [16]:
# check the missing values/text 
twitter.isnull().sum() 
#There are no missing values

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [17]:
# check +ve and -ve tweets present in dta frame
twitter['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [18]:
#converting 4 as 1
twitter['target'] = twitter['target'].replace(4, 1)

In [19]:
twitter['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative 
1 --> Positive

Stemming -->
process of reducing a word to root word

In [20]:
port_stem = PorterStemmer()

In [23]:
def stemming(content):
    stemmed_content = re.sub("[^a-zA-Z]", " ", content)  # Remove non-alphabetic characters
    stemmed_content = stemmed_content.lower()           # Convert to lowercase
    stemmed_content = stemmed_content.split()           # Split into words
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]  # Stem words and remove stopwords
    stemmed_content = " ".join(stemmed_content)         # Join the words back into a string
    return stemmed_content

In [ ]:
# Apply the function to a DataFrame column
twitter['stemmed_content'] = twitter['text'].apply(stemming)

In [ ]:
twitter.head()

In [ ]:
print(twitter['stemmed_content'])

In [ ]:
print(twitter['target'])

In [ ]:
# seperating the data and the lable

X = twitter['stemmed_content'].values
Y = twitter['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

Splitting the data into training and testing data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

20% of data is send for training 

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
# converting the text data into neumerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

Training the ML model

In [ ]:
# logistic regression 

model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

Model Evaluation

In [ ]:
# accuracy score on training data 
 
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)


In [ ]:
# printing the training data accuracy

print('Accuracy score on the training data :', training_data_accuracy)

In [ ]:
# accuracy score on test data 
 
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
# printing the test data accuracy

print('Accuracy score on the test data :', test_data_accuracy)

How we can save this model and how we can use it in future 

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future predictions 

In [ ]:
# loading the saved model

loaded_model = pickle.load(open('   ', 'rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print('Negative Tweet')

else:
    print('Positive Tweet')

In [ ]:
X_new = X_test[30]
print(Y_test[30])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
    print('Negative Tweet')

else:
    print('Positive Tweet')